In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hsk_html_parser

In [ ]:
with open("data/html_files/HSK_6.html") as f:
    html_file = "".join(f.readlines())

# Creating Flashcards

## English

In [ ]:
parser = hsk_html_parser.HskHtmlParser()
parser.feed(html_file)

In [ ]:
# Classic HSK decks with automatic pinyin detection
output_word = f"xml_outputs/English/automatic_pinyin/HSK_{parser.content['hskLevel']}_word_list.xml"
parser.create_word_xml_automatic(output_word)
output_sentence = f"xml_outputs/English/automatic_pinyin/HSK_{parser.content['hskLevel']}_sentence_list.xml"
parser.create_sentence_xml_automatic(output_sentence)

# Custom HSK decks with manual pinyin implementation
output_word = f"xml_outputs/English/HSK_{parser.content['hskLevel']}_word_list.xml"
parser.create_word_xml(output_word)
output_sentence = f"xml_outputs/English/HSK_{parser.content['hskLevel']}_sentence_list.xml"
parser.create_sentence_xml(output_sentence)

## French

In [ ]:
parser_french = hsk_html_parser.HskHtmlParser(translate_to_french=True, u8_file="data/cfdict.u8")
parser_french.feed(html_file)

In [ ]:
# Classic HSK decks with automatic pinyin detection
output_word = f"xml_outputs/French/automatic_pinyin/HSK_{parser_french.content['hskLevel']}_word_list.xml"
parser_french.create_word_xml_automatic(output_word)

# Custom HSK decks with manual pinyin implementation
output_word = f"xml_outputs/French/HSK_{parser_french.content['hskLevel']}_word_list.xml"
parser_french.create_word_xml(output_word)